# CrysFML: Xrays PbSO4

In [1]:
# Misc
import sys, os
import numpy as np

# CrysFML
import CFML_api

# Vizualization
import py3Dmol
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

In [2]:
output_notebook()
FIGURE_WIDTH = 700
FIGURE_HEIGHT = 300

Loading BokehJS ...

In [3]:
def diffraction_pattern_xy(space_group, cell, atom_list, job_info):
    reflection_list = CFML_api.ReflectionList(cell, space_group, 
                                              True, 
                                              job_info)
    reflection_list.compute_structure_factors(space_group, 
                                              atom_list, 
                                              job_info)
    diffraction_pattern = CFML_api.DiffractionPattern(job_info,
                                                  reflection_list, 
                                                  cell.reciprocal_cell_vol)
    return diffraction_pattern.x, diffraction_pattern.ycalc

## Define sample model

### Create space group object

In [4]:
space_group = CFML_api.SpaceGroup('P n m a')
#space_group.print_description()

### Create unit cell object

In [5]:
lengths = np.asarray([8.485680,5.402480,6.964544], dtype='float32')
angles = np.asarray([90,90,90], dtype='float32')
unit_cell = CFML_api.Cell(lengths, angles)
#unit_cell.print_description()

### Create list of atoms

In [6]:
cif_str = '''
loop_
 _atom_site_label
 _atom_site_type_symbol
 _atom_site_occupancy
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_adp_type
 _atom_site_U_iso_or_equiv
  Pb  Pb  1.0  0.18797  0.25    0.16754  Uiso  0.0173
  S   S   1.0  0.06467  0.25    0.68300  Uiso  0.0115
  O1  O   1.0  0.90705  0.25    0.59672  Uiso  0.0070
  O2  O   1.0  0.18641  0.25    0.54269  Uiso  0.0127
  O3  O   1.0  0.08025  0.02961 0.81208  Uiso  0.0135
'''
atom_list = CFML_api.AtomList(cif_str.split('\n'))
atom_list.set_mult_occ_cif(space_group)
atom_list.print_description()


        Atoms information:
        ------------------

    Atom      Chem        x/a       y/b       z/c       Biso     Occ       Mult
    Pb         Pb       0.1880    0.2500    0.1675    1.3660    0.5000        4  None
    S          S        0.0647    0.2500    0.6830    0.9080    0.5000        4  None
    O1         O        0.9071    0.2500    0.5967    0.5527    0.5000        4  None
    O2         O        0.1864    0.2500    0.5427    1.0028    0.5000        4  None
    O3         O        0.0803    0.0296    0.8121    1.0659    1.0000        8  None


## Load experimental data

### Load measured data

In [7]:
fpath_obs = '../datasets/xrays_powder_PbSO4/XPD.xye'
x_obs, y_obs, sy_obs = np.loadtxt(fpath_obs, unpack=True)

### Visualize measured data

In [8]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_obs, y_obs, legend_label='Iobs', color='darkgray', line_width=2)
show(fig)

## Analyse data

### Create job definition object

In [9]:
cfl_str = '''
Title PbSO4
Patt_1 XRAY_2THE  1.5405    1.5444    1.00      0.0        135.0
'''
job_info = CFML_api.JobInfo(cfl_str.split('\n'))
job_info.print_description()

PBSO4
Number of patterns: 1
Type of pattern: XRAY_2THE       
Number of phases: 1
Name of phases: PBSO4                                                                                                                           
Name of phases: PBSO4                                                                                                                           
Lambda range: (1.5405000448226929, 1.5443999767303467)
Lambda ratio: 1.0
Range 2theta: (0.0, 135.0)
Range sin(theta)/lambda: (0.0, 0.5982125997543335)


### Modify job definition object

In [10]:
job_info.pattern_type = "XRAY_2THE"
job_info.range_2theta = (10.0, 160.0)
job_info.theta_step = 0.025
#job_info.lambdas = (1.5405, 1.5444)
job_info.lambda_ratio = 0.5
job_info.u_resolution = 0.031112
job_info.v_resolution = -0.052102
job_info.w_resolution = 0.032035
job_info.x_resolution = 0.14710 # 0.014710
job_info.y_resolution = 0.0
#job_info.u_resolution = 0.050944
#job_info.v_resolution = -0.099166
#job_info.w_resolution = 0.039486
#job_info.x_resolution = 0.149145
job_info.print_description()

PBSO4
Number of patterns: 1
Type of pattern: XRAY_2THE       
Number of phases: 1
Name of phases: PBSO4                                                                                                                           
Name of phases: PBSO4                                                                                                                           
Lambda range: (1.5405000448226929, 1.5443999767303467)
Lambda ratio: 0.5
Range 2theta: (10.0, 160.0)
Range sin(theta)/lambda: (0.05657626688480377, 0.6376636624336243)


### Calculate and plot diffraction pattern

In [11]:
x_calc, y_clac = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)

 XRAY_2THE       


In [12]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_calc, y_clac, legend_label='Icalc', color='orangered', line_width=1)
show(fig)

### Compare observed and calculated data

In [13]:
scale = 0.4
background = 100
zero_shift = -0.0

In [14]:
x_calc, y_calc = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)
x_calc = x_calc + zero_shift
y_calc = y_calc * scale + background

 XRAY_2THE       


In [15]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_obs, y_obs, legend_label='Iobs', color='darkgray', line_width=2)
fig.line(x_calc, y_calc, legend_label='Icalc', color='orangered', line_width=1)
show(fig)

In [16]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT, x_range=(20,30))
fig.line(x_obs, y_obs, legend_label='Iobs', color='darkgray', line_width=2)
fig.line(x_calc, y_calc, legend_label='Icalc', color='orangered', line_width=1)
show(fig)

In [17]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT, x_range=(43,46.5))
fig.line(x_obs, y_obs, legend_label='Iobs', color='darkgray', line_width=2)
fig.line(x_calc, y_calc, legend_label='Icalc', color='orangered', line_width=1)
show(fig)

In [18]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT, x_range=(61,65))
fig.line(x_obs, y_obs, legend_label='Iobs', color='darkgray', line_width=2)
fig.line(x_calc, y_calc, legend_label='Icalc', color='orangered', line_width=1)
show(fig)

## Test lambda ratio

In [19]:
job_info.lambda_ratio = 1.0
_, y_calc_1_0 = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)
job_info.lambda_ratio = 0.0
_, y_calc_0_0 = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)
job_info.lambda_ratio = 0.5
_, y_calc_0_5 = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)

 XRAY_2THE       
 XRAY_2THE       
 XRAY_2THE       


In [20]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT, x_range=(43.4,44.2))
fig.line(x_obs, y_obs, legend_label='Iobs', color='darkgray', line_width=2)
fig.line(x_calc, y_calc_1_0, legend_label='lambda ratio: 1.0', color='red', line_width=10)
fig.line(x_calc, y_calc_0_0, legend_label='lambda ratio: 0.0', color='blue', line_width=6)
fig.line(x_calc, y_calc_0_5, legend_label='lambda ratio: 0.5', color='yellow', line_width=2)
show(fig)